# **FGV EESP**

- ## Dados Econômicos - Brasil
---



### **Bibliotecas**

In [ ]:
# Obter os dados do SGS do Banco Central do Brasil
install.packages("rbcb")

# Salvar dados em formato .xlsx
install.packages("writexl")

# Implementar filtro HP
install.packages("mFilter")

In [ ]:
library(dplyr)
library(httr)
library(jsonlite)
library(lubridate)
library(mFilter)
library(readxl)
library(rbcb)
library(tidyverse)
library(writexl)
library(zoo)

### **Dados**

- #### **Expectativa de Inflação do Mercado**

  * Referência: [Olinda BCB](https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/aplicacao#!/recursos/ExpectativasMercadoInflacao12Meses#eyJmb3JtdWxhcmlvIjp7IiRmb3JtYXQiOiJqc29uIiwiJHRvcCI6MTAwLCIkZmlsdGVyIjoiBDkEIGVxIDAgYW5kIAQyBCBlcSAnUycgYW5kIAQwBCBlcSAnSVBDQScgYW5kIAQxBCBnZSAnMjAwNi0wMS0wMScgYW5kIAQxBCBsZSAnMjAyMy0wNi0zMCciLCIkb3JkZXJieSI6IgQxBCBkZXNjIn0sInByb3ByaWVkYWRlcyI6WzAsMSwzLDQsNV0sInBlc3F1aXNhZG8iOnRydWUsImFjdGl2ZVRhYiI6InRhYmxlIiwiZ3JpZFN0YXRlIjp7AzADOlt7A0IDIgQwBCIsA0EDfSx7A0IDIgQxBCIsA0EDfSx7A0IDIgQzBCIsA0EDfSx7A0IDIgQzBG5hIiwDQQN9LHsDQgMiBDUEIiwDQQN9XSwDMQM6e30sAzIDOltdLAMzAzp7fSwDNAM6e30sAzUDOnt9fSwicGl2b3RPcHRpb25zIjp7A2EDOnt9LANiAzpbXSwDYwM6NTAwLANkAzpbIgQxBCJdLANlAzpbXSwDZgM6WyJN6WRpYSJdLANnAzoidmFsdWVfel90b19hIiwDaAM6InZhbHVlX3pfdG9fYSIsA2kDOnt9LANqAzp7fSwDawM6ODUsA2wDOmZhbHNlLANtAzp7fSwDbgM6e30sA28DOiJN6WRpYSIsA3ADOiJMaW5lIENoYXJ0In19)

In [ ]:
# Definir a URL da API do BCB para obter dados de Expectativa do Mercado para IPCA nos próximos 12 meses
url <- "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoInflacao12Meses?$filter=baseCalculo%20eq%200%20and%20Suavizada%20eq%20'S'%20and%20Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2005-12-01'%20and%20Data%20le%20'2023-08-31'&$orderby=Data%20desc&$format=json&$select=Indicador,Data,Media,Mediana,DesvioPadrao"

# Realizar a consulta e ler o arquivo JSON
response <- GET(url)
data_json <- rawToChar(response$content)

# Ajuste das colunas de data
focus_inflation_expectation <- fromJSON(data_json)$value %>%
  mutate(day = ymd(Data),
         month_date = floor_date(day, unit = "month"),
         year = year(month_date),
         day_of_week = weekdays(day),
         month_name = month(month_date, label = TRUE, abbr = TRUE))

# Filtrar apenas as sextas-feiras (Referência do Boletim Focus)
df_fridays_only <- focus_inflation_expectation %>%
  filter(day_of_week == "Friday")

# Calcular média da coluna "Mediana" agrupada por "year" e "month_date"
df_olinda <- as.data.frame(df_fridays_only %>%
  group_by(year, month_date, month_name) %>%
  summarize(market_inflation_expectation = round(mean(Mediana), 2)))

# Converter variável em série temporal
df_olinda$market_inflation_expectation <- ts(df_olinda$market_inflation_expectation, start = c(2005, 12), frequency = 12)

# Adicionar a coluna 'month' com base em 'market_inflation_expectation'
df_olinda$month <- time(df_olinda$market_inflation_expectation)

- #### **Meta de Inflação**

  * Referência: [BCB](https://www.bcb.gov.br/controleinflacao/historicometas)

In [4]:
# Histórico das metas para a inflação

# Caminho do arquivo original
file_path <- "https://github.com/matheusrmorgado/inflation-expectation/raw/main/data/bcb_inflation_target.xlsx"

# Download do arquivo
download.file(file_path, "bcb_inflation_target.xlsx")

# Ler o arquivo Excel e converter em dataframe
df_bcb <- as.data.frame(read_excel("bcb_inflation_target.xlsx"))

# Filtrar os dados de 2005-12-01 até 2023-08-01
df_bcb <- df_bcb %>% filter(as.Date(month_date) >= as.Date("2005-12-01") & as.Date(month_date) <= as.Date("2023-08-01"))

# Lista de variáveis
variables_to_convert <- c("inflation_target","inflation_target_lower_bound","inflation_target_upper_bound","inflation_target_wtd_avg","inflation_target_lower_bound_wtd_avg","inflation_target_upper_bound_wtd_avg")

# Loop para converter as variáveis em séries temporais
for (variable in variables_to_convert) {
  df_bcb[[variable]] <- ts(df_bcb[[variable]], start = c(2005, 12), frequency = 12)
}

# Selecionar apenas variáveis relacionadas a inflação
df_bcb <- df_bcb[, 3:8]

- #### **Expectativa de Inflação dos Consumidores**

  * Referência: [FGV IBRE](https://extra-ibre.fgv.br/ibre/sitefgvdados/default.aspx?Convidado=S)

In [5]:
# Indicador de Expectativa de Inflação dos Consumidores - Código FGV Dados: 1416188
# Índice de Confiança do Consumidor com Ajuste Sazonal - Código FGV Dados: 1006982

# Caminho do arquivo original
file_path <- "https://github.com/matheusrmorgado/inflation-expectation/raw/main/data/fgv_ibre.xlsx"

# Download do arquivo
download.file(file_path, "fgv_ibre.xlsx")

# Ler o arquivo Excel e converter em dataframe
df_fgv <- as.data.frame(read_excel("fgv_ibre.xlsx"))

# Filtrar os dados de 2005-12-01 até 2023-08-01
df_fgv <- df_fgv %>% filter(as.Date(month_date) >= as.Date("2005-12-01") & as.Date(month_date) <= as.Date("2023-08-01"))

# Conversão em formato timeseries
df_consumer <- as.data.frame(ts(df_fgv$consumer_inflation_expectation, start = c(2005, 12), frequency = 12))
df_consumer <- rename(df_consumer, consumer_inflation_expectation = x)

df_confidence <- as.data.frame(ts(df_fgv$consumer_confidence_index, start = c(2005, 12), frequency = 12))
df_confidence <- rename(df_confidence, consumer_confidence_index = x)

- #### **Dados Macroeconômicos**

  * Referência: [SGS BCB](https://www3.bcb.gov.br/sgspub/)

In [6]:
# Índice nacional de preços ao consumidor amplo acumulado em 12 meses (%)
# SGS: 13522
df_ipca <- as.data.frame(get_series(13522, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")) %>% rename(ipca = x)

# Taxa de juros Selic acumulada no mês anualizada base 252 (% a.a.)
# SGS: 4189
df_selic <- as.data.frame(get_series(4189, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")) %>% rename(selic = x)

# Selic deflacionada pelo IPCA
df_selic_deflated <- 100*(((1+df_selic/100)/(1+df_ipca/100))-1) %>% rename(selic_deflated = selic)

# PIB acumulado dos últimos 12 meses - valores correntes
# SGS: 4382
df_pib <- as.data.frame(get_series(4382, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")) %>% rename(gdp = x)

# Divida líquida do governo geral - Saldos em R$ milhões
# SGS: 4501
df_primary_fiscal_result <- as.data.frame(get_series(4501, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")) %>% rename(primary_fiscal_result = x)

# Dívida líquida do governo geral (% PIB)
# SGS: 4536
df_primary_fiscal_result_to_gdp <- as.data.frame(get_series(4536, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")) %>% rename(primary_fiscal_result_to_gdp = x)
df_primary_fiscal_result_to_gdp <- df_primary_fiscal_result_to_gdp/100

In [7]:
# Índice da taxa de câmbio real efetiva (IPCA) - Jun/1994=100
# SGS: 11752
df_real_exchange_rate <- as.data.frame(get_series(11752, start_date = "2004-01-01", end_date = "2023-08-01")) %>%
  rename(real_exchange_rate = "11752") %>%
  filter(date >= as.Date("2005-12-01")) %>%
  select(real_exchange_rate)

df_real_exchange_rate$real_exchange_rate <- ts(df_real_exchange_rate$real_exchange_rate, start = c(2005, 12), frequency = 12)
df_real_exchange_rate$real_exchange_rate_trend <- as.numeric(hpfilter(df_real_exchange_rate$real_exchange_rate)$trend)
df_real_exchange_rate$real_exchange_rate_trend <- ts(df_real_exchange_rate$real_exchange_rate_trend, start = c(2005, 12), frequency = 12)
df_real_exchange_rate$real_exchange_rate_gap <- 100*((df_real_exchange_rate$real_exchange_rate/df_real_exchange_rate$real_exchange_rate_trend) - 1)

In [8]:
# Fator Deflacionário: Renda Nacional Disponível Bruta das Famílias, em valores constantes, deflacionada pelo IPCA (média móvel trimestral)	/ Renda Nacional Disponível Bruta das Famílias - em valores correntes (média móvel trimestral)
# SGS: 29025 / 29023
df_deflation_factor <- get_series(29025, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")/
  get_series(29023, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts")

# Renda Nacional Disponível Bruta das Famílias - em valores correntes, acumulada em doze meses
# SGS: 29029
df_rndbf_deflated <- as.data.frame(get_series(29029, start_date = "2005-12-01", end_date = "2023-08-01", as = "ts") * df_deflation_factor) %>% rename(real_household_income = x)

df_rndbf_deflated$real_household_income <- ts(df_rndbf_deflated$real_household_income, start = c(2005, 12), frequency = 12)
df_rndbf_deflated$real_household_income_trend <- as.numeric(hpfilter(df_rndbf_deflated$real_household_income)$trend)
df_rndbf_deflated$real_household_income_trend <- ts(df_rndbf_deflated$real_household_income_trend, start = c(2005, 12), frequency = 12)
df_rndbf_deflated$real_household_income_gap <- 100*((df_rndbf_deflated$real_household_income/df_rndbf_deflated$real_household_income_trend) - 1)

- #### **União de todas as bases de dados**

In [9]:
# Realiza a unificação de todas as tabelas do BCB e FGV IBRE
df_public_data <- cbind(df_olinda, df_consumer, df_confidence, df_bcb, df_ipca, df_selic, df_selic_deflated, df_rndbf_deflated, df_real_exchange_rate, df_pib, df_primary_fiscal_result, df_primary_fiscal_result_to_gdp)

# Calcula o lag para IPCA e Selic
df_public_data$ipca_lag <- c(NA, df_public_data$ipca[-nrow(df_public_data)])
df_public_data$selic_deflated_lag <- c(NA, df_public_data$selic_deflated[-nrow(df_public_data)])
df_public_data$primary_fiscal_result_to_gdp_lag <- c(NA, df_public_data$primary_fiscal_result_to_gdp[-nrow(df_public_data)])
df_public_data$real_exchange_rate_gap_lag <- c(NA, df_public_data$real_exchange_rate_gap[-nrow(df_public_data)])
df_public_data$real_household_income_gap_lag <- c(NA, df_public_data$real_household_income_gap[-nrow(df_public_data)])
df_public_data$consumer_confidence_index_lag <- c(NA, df_public_data$consumer_confidence_index[-nrow(df_public_data)])

# Converter as colunas "ipca_lag" e "selic_lag" para séries temporais
df_public_data$ipca_lag <- ts(df_public_data$ipca_lag, start = c(2005, 12), frequency = 12)
df_public_data$selic_deflated_lag <- ts(df_public_data$selic_deflated_lag, start = c(2005, 12), frequency = 12)
df_public_data$primary_fiscal_result_to_gdp_lag <- ts(df_public_data$primary_fiscal_result_to_gdp_lag, start = c(2005, 12), frequency = 12)
df_public_data$real_exchange_rate_gap_lag <- ts(df_public_data$real_exchange_rate_gap_lag, start = c(2005, 12), frequency = 12)
df_public_data$real_household_income_gap_lag <- ts(df_public_data$real_household_income_gap_lag, start = c(2005, 12), frequency = 12)
df_public_data$consumer_confidence_index_lag <- ts(df_public_data$consumer_confidence_index_lag, start = c(2005, 12), frequency = 12)

# Reorganiza as colunas, colocando "month_date" e "month" nas primeiras posições
df_public_data <- df_public_data %>% dplyr::select(year, month_name, month_date, month, everything())

In [10]:
# Identificar as variáveis que são Time-Series
ts_cols <- sapply(df_public_data, is.ts)

# Filtrar o dataframe
df_public_data <- df_public_data %>%
  filter(month_date >= as.Date("2006-01-01"))

# Transformar as colunas identificadas como Time-Series de volta em Time-Series
df_public_data[ts_cols] <- lapply(df_public_data[ts_cols], function(x) ts(x, start = c(2006, 1), frequency = 12))

- #### **Visualização da base de dados**

In [11]:
str(df_public_data)

'data.frame':	212 obs. of  31 variables:
 $ year                                : num  2006 2006 2006 2006 2006 ...
 $ month_name                          : Ord.factor w/ 12 levels "Jan"<"Feb"<"Mar"<..: 1 2 3 4 5 6 7 8 9 10 ...
 $ month_date                          : Date, format: "2006-01-01" "2006-02-01" ...
 $ month                               : Time-Series  from 2006 to 2024: 2006 2006 2006 2006 2006 ...
 $ market_inflation_expectation        : Time-Series  from 2006 to 2024: 4.58 4.49 4.36 4.16 4.18 4.25 4.4 4.48 4.29 4.05 ...
 $ consumer_inflation_expectation      : Time-Series  from 2006 to 2024: 6.9 7 6.9 6.5 6.7 6.3 6.9 6.4 6.3 5.8 ...
 $ consumer_confidence_index           : Time-Series  from 2006 to 2024: 93.3 92.8 93 91.8 90.7 90.7 91.2 91.8 91.5 94.3 ...
 $ inflation_target                    : Time-Series  from 2006 to 2024: 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 4.5 ...
 $ inflation_target_lower_bound        : Time-Series  from 2006 to 2024: 2.5 2.5 2.5 2.5 2.5 2.5 2.5 2

In [12]:
head(df_public_data, 5)

,year,month_name,month_date,month,market_inflation_expectation,consumer_inflation_expectation,consumer_confidence_index,inflation_target,inflation_target_lower_bound,inflation_target_upper_bound,⋯,real_exchange_rate_gap,gdp,primary_fiscal_result,primary_fiscal_result_to_gdp,ipca_lag,selic_deflated_lag,primary_fiscal_result_to_gdp_lag,real_exchange_rate_gap_lag,real_household_income_gap_lag,consumer_confidence_index_lag
,<dbl>,<ord>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2006,Jan,2006-01-01,2006.000,4.58,6.9,93.3,4.5,2.5,6.5,⋯,2.311963,2190643,1022618,0.4668,5.69,11.87435,0.4654,2.145933,2.0099366,88.7
2,2006,Feb,2006-02-01,2006.083,4.49,7.0,92.8,4.5,2.5,6.5,⋯,-2.641655,2207671,1023884,0.4638,5.70,11.30558,0.4668,2.311963,1.8753112,93.3
3,2006,Mar,2006-03-01,2006.167,4.36,6.9,93.0,4.5,2.5,6.5,⋯,-2.421135,2225145,1032060,0.4638,5.51,11.15534,0.4638,-2.641655,1.3150849,92.8
4,2006,Apr,2006-04-01,2006.250,4.16,6.5,91.8,4.5,2.5,6.5,⋯,-2.198977,2235813,1019291,0.4559,5.32,10.84314,0.4638,-2.421135,0.8411086,93.0
5,2006,May,2006-05-01,2006.333,4.18,6.7,90.7,4.5,2.5,6.5,⋯,2.482859,2256419,1042621,0.4621,4.63,11.04846,0.4559,-2.198977,0.5015829,91.8


- #### **Salvar em formato `.csv` e `.xlsx`**

In [13]:
# Salve o DataFrame como um arquivo CSV localmente
write.csv(df_public_data, file = "public_data.csv", row.names = FALSE)

In [14]:
# Salve o DataFrame como um arquivo XLSX localmente
write_xlsx(df_public_data, path = "/content/public_data.xlsx")